In [2]:
suppressMessages(library(rwwa))
png_res <- 240

# Check results using original functions

In [23]:
nsamp <- 1000

varnm <- "rx1day-ondjfm"
rnm <- "s"

# load covariates
gmst <- read.table("ts-obs/gmst.dat", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2026]
nao <- load_ts("ts-obs/med-storms_nao-djf_era5-stn.dat", col.names = c("year", "nao"))

cov_df <- merge(gmst, nao, all.x = T)
cov_df$gmst <- cov_df$gmst - cov_df$gmst[cov_df$year == 2026]

# define factual & counterfactual climates
cov_2026 <- cov_df[cov_df$year == 2026,c(-1),drop = F]
cov_cf <- rbind("pi" = cov_2026 - c(1.3,0),
                "naoneutral" = c(cov_2026$gmst, 0),
                "pineutral" = c(-1.3, 0))

era5 <- merge(cov_df, load_ts(paste0("ts-obs/med-storms_",varnm,"_",rnm,"_era5.dat"), col.names = c("year", "x"))) # 1950-2026
eobs <- merge(cov_df, load_ts(paste0("ts-obs/med-storms_",varnm,"_",rnm,"_eobs.dat"), col.names = c("year", "x"))) # 1920-2026

mdl_era5_gmstnao <- fit_ns("gev", "fixeddisp", data = era5, covnm = c("gmst", "nao"), varnm = "x", lower = F)
mdl_era5_gmst <- fit_ns("gev", "fixeddisp", data = era5, covnm = c("gmst"), varnm = "x", lower = F)

mdl_eobs_gmstnao <- fit_ns("gev", "fixeddisp", data = eobs[!is.na(eobs$na),], covnm = c("gmst", "nao"), varnm = "x", lower = F)
mdl_eobs_gmst1920 <- fit_ns("gev", "fixeddisp", data = eobs, covnm = c("gmst"), varnm = "x", lower = F)
mdl_eobs_gmst1950 <- fit_ns("gev", "fixeddisp", data = eobs[!is.na(eobs$na),], covnm = c("gmst"), varnm = "x", lower = F)

In [24]:
mlist <- list("ERA5 - GMST only" = mdl_era5_gmst,
            "ERA5 - GMST + NAO" = mdl_era5_gmstnao,
            "EObs - GMST only (1920-2026)" = mdl_eobs_gmst1920,
            "EObs - GMST only (1950-2026)" = mdl_eobs_gmst1950,
            "EObs - GMST + NAO" = mdl_eobs_gmstnao)

res <- rbind.fill(sapply(mlist, function(mdl) data.frame(t(mdl_ests(mdl, cov_f = cov_2026, cov_cf = cov_cf))), simplify = F))
rownames(res) <- names(mlist)

In [25]:
res[,c("return_period","PR_pi","dI_rel_pi")]
# E-Obs results fairly stable regardless of whether using 1920 or 1950 onwards

,return_period,PR_pi,dI_rel_pi
,<dbl>,<dbl>,<dbl>
ERA5 - GMST only,82.613075,52.789138,17.65778
ERA5 - GMST + NAO,39.799635,64.942525,23.00887
EObs - GMST only (1920-2026),2.998259,6.873176,38.15117
EObs - GMST only (1950-2026),2.734766,7.319387,42.38718
EObs - GMST + NAO,2.223773,8.818264,49.21544


## Get results for RP40 event

In [26]:
mlist_rp40 <- lapply(mlist, function(mdl) {
    rl40 <- eff_return_level(mdl, rp = 40, fixed_cov = cov_2026)
    fit_ns(dist = mdl$dist, type = mdl$type, data = mdl$data, varnm = mdl$varnm, covnm = mdl$covnm, lower = mdl$lower, ev = rl40, ev_year = 2026)
})

In [27]:
res40 <- rbind.fill(sapply(mlist_rp40, function(mdl) data.frame(t(mdl_ests(mdl, cov_f = cov_2026, cov_cf = cov_cf))), simplify = F))
rownames(res40) <- names(mlist_rp40)

In [28]:
res[,c("return_period","PR_pi","dI_rel_pi")]        # print the original results
res40[,c("return_period","PR_pi","dI_rel_pi")]
# E-Obs results fairly stable regardless of whether using 1920 or 1950 onwards

,return_period,PR_pi,dI_rel_pi
,<dbl>,<dbl>,<dbl>
ERA5 - GMST only,82.613075,52.789138,17.65778
ERA5 - GMST + NAO,39.799635,64.942525,23.00887
EObs - GMST only (1920-2026),2.998259,6.873176,38.15117
EObs - GMST only (1950-2026),2.734766,7.319387,42.38718
EObs - GMST + NAO,2.223773,8.818264,49.21544


,return_period,PR_pi,dI_rel_pi
,<dbl>,<dbl>,<dbl>
ERA5 - GMST only,40,21.57370,17.65778
ERA5 - GMST + NAO,40,65.42776,23.00887
EObs - GMST only (1920-2026),40,26.32044,38.15117
EObs - GMST only (1950-2026),40,29.46276,42.38718
EObs - GMST + NAO,40,48.41232,49.21544


In [29]:
boot_all <- lapply(mlist_rp40, boot_ci, cov_f = cov_2026, cov_cf = cov_cf, nsamp = nsamp)